# Model Training
This notebook is used for training purposes of training and saving fine-tuned models on different size datasets and hiperparameters

## Load Base Model

Load base DistilBERT uncased model that will be used as a starting pretrained model for fine-tuning models


In [1]:
from transformers import AutoModel
import torch

In [3]:
base_model = AutoModel.from_pretrained("bert-base-uncased", torch_dtype=torch.float32)

Import necessary libraries for model training 

## Prepare data for training

Import necessary libraries and set global parameters for training and saving model

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from transformers import AutoModel
from models.ModelRetriever import get_full_classification_model, get_classification_head_model, get_adapters_model, get_lora_model
from hf_utils import save_model_to_hf
import json
import time
import pickle
from sklearn.metrics import accuracy_score, f1_score
from datasets import load_from_disk
from torch import nn

In [ ]:
METRICS_DIR = './training/training_metrics'
TRAINER_DIR = './training/trainer_metrics'
SEED = 42

LR = 2e-4
EPOCHS = 10 
NUM_LABELS = 2 # Binary classification problem
MAX_LENGTH = 512 # Max length of tokens for examples, if less pad with 0, if more truncate
BATCH_SIZE=32 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

Paths to vectorized datasets ready for models to use <br>
If empty, run Dataset_Loader.ipynb notebook

In [ ]:
DATASETS_PATHS = {
    'small': {
        'train': './datasets/train_dataset_small',
        'validation': './datasets/validation_dataset_small',
        'test': './datasets/test_dataset'
    },
    'medium': {
        'train': './datasets/train_dataset_medium',
        'validation': './datasets/validation_dataset_medium',
        'test': './datasets/test_dataset'
    },
    'full': {
        'train': './datasets/full_train',
        'validation': './datasets/full_validation',
        'test': './datasets/test_dataset'
    }
}

## Functions for training process

For performing and monitoring training processes, also contains functions for saving results

Monitor training metrics, such as time for each epoch and gpu memory usage

In [ ]:
class TrainingLogger:
    """
    Tracks per-epoch time and GPU memory usage.
    """
    def __init__(self):
        self.epoch_times = []
        self.memory_log = []
        self.start_time = None
        self.total_training_time = None

    def start_training(self):
        self.start_time = time.time()

    def end_training(self):
        self.total_training_time = time.time() - self.start_time

    def log_epoch(self, epoch, epoch_start_time):
        # time
        epoch_time = time.time() - epoch_start_time
        self.epoch_times.append({"epoch": epoch + 1, "time_seconds": epoch_time})

        # GPU mem
        if torch.cuda.is_available():
            mem_mb = torch.cuda.memory_allocated(0) // (1024 ** 2)
            self.memory_log.append({"epoch": epoch + 1, "gpu_memory_used_mb": int(mem_mb)})

Monitor training metrics for later analisys

In [ ]:
class ModelTrainer:
    def __init__(self, model: nn.Module, device: str = 'cpu'):
        self.model = model.to(device)
        self.device = device

    def train(
        self,
        train_loader: DataLoader,
        val_loader: DataLoader,
        optimizer: torch.optim.Optimizer,
        loss_fn: nn.Module,
        epochs: int,
        save_dir: TRAINER_DIR,
        filename: str,
        num_labels: int = 2,
        grad_clip: float = 1.0
    ):
        os.makedirs(save_dir, exist_ok=True)

        logger = TrainingLogger()
        logger.start_training()

        history = {"train_loss": [], "val_loss": [], "val_accuracy": [], "val_f1": []}

        for epoch in range(epochs):
            epoch_start = time.time()
            self.model.train()
            running_loss = 0.0

            for batch in train_loader:
                input_ids     = batch["input_ids"].to(self.device)
                attention_mask= batch["attention_mask"].to(self.device)
                labels        = batch["label"].to(self.device)

                optimizer.zero_grad(set_to_none=True)
                logits = self.model(input_ids, attn_mask=attention_mask)
                loss = loss_fn(logits, labels)
                loss.backward()

                if grad_clip is not None:
                    torch.nn.utils.clip_grad_norm_(
                        (p for p in self.model.parameters() if p.requires_grad), grad_clip
                    )

                optimizer.step()

                running_loss += loss.item()

            avg_train_loss = running_loss / max(1, len(train_loader))
            history["train_loss"].append(avg_train_loss)

            # ---- validation ----
            self.model.eval()
            val_loss = 0.0
            all_preds, all_labels = [], []
            with torch.no_grad():
                for batch in val_loader:
                    input_ids      = batch["input_ids"].to(self.device)
                    attention_mask = batch["attention_mask"].to(self.device)
                    labels         = batch["label"].to(self.device)

                    logits = self.model(input_ids, attn_mask=attention_mask)
                    loss = loss_fn(logits, labels)
                    val_loss += loss.item()

                    preds = torch.argmax(logits, dim=1).cpu()
                    all_preds.extend(preds.tolist())
                    all_labels.extend(labels.cpu().tolist())

            avg_val_loss = val_loss / max(1, len(val_loader))
            val_acc = accuracy_score(torch.tensor(all_labels), torch.tensor(all_preds))
            val_f1 = f1_score(all_labels, all_preds, average="weighted", labels=list(range(num_labels)))
            
            history["val_loss"].append(avg_val_loss)
            history["val_accuracy"].append(val_acc)
            history["val_f1"].append(val_f1)

            logger.log_epoch(epoch, epoch_start)

            print(
                f"Epoch {epoch+1}/{epochs} | "
                f"Train Loss: {avg_train_loss:.4f} | "
                f"Val Loss: {avg_val_loss:.4f} | "
                f"Val Acc: {val_acc:.4f} | "
                f"Val F1: {val_f1:.4f}"
            )

        logger.end_training()

        # persist logs
        log_blob = {
            "history": history,
            "epoch_times": logger.epoch_times,
            "gpu_memory_log": logger.memory_log,
            "total_training_time_seconds": logger.total_training_time,
        }
        
        file_path = os.path.join(save_dir, f"{filename}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(log_blob, f)

        print(f"Saved metrics to: {file_path}")
        return history

Start training process for each model <br>
Returns fine-tuned model

In [ ]:
def Train(model, size, name):
    print(f"\n========== Training {name} ==========")
    save_dir = os.path.join(METRICS_DIR, name)

    train_ds = load_from_disk(DATASETS_PATHS[size]['train'])
    val_ds = load_from_disk(DATASETS_PATHS[size]['validation'])

    train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)     

    # optimizer only on trainable params
    optim = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=LR)
    loss_fn = nn.CrossEntropyLoss()

    # log param counts
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params     = sum(p.numel() for p in model.parameters())
    os.makedirs(os.path.join(save_dir), exist_ok=True)
    with open(os.path.join(save_dir, "run_config.json"), "w") as f:
        json.dump({
            "model_name": name,
            "epochs": EPOCHS,
            "lr": LR,
            "num_labels": NUM_LABELS,
            "max_length": MAX_LENGTH,
            "trainable_params": int(trainable_params),
            "total_params": int(total_params),
            "device": DEVICE
        }, f, indent=2)

    # Train model
    trainer = ModelTrainer(model, device=DEVICE)
    trainer.train(
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optim,
        loss_fn=loss_fn,
        epochs=EPOCHS,
        save_dir=save_dir,
        filename=name,
        num_labels=NUM_LABELS,
        grad_clip=1.0
    )



## Train models

Each function will take base model, transform it to specific model suitable for different fine-tuning purpose and run execution on small, medium and full dataset. Returned model is saved on Hugging Face platform 

### Full Fine-Tuning
Run execution on small, medium and full dataset. Save results on HF platform

In [ ]:
for size in ['small', 'medium', 'full']:
        name = f"full_fine_tuning_set_{size}"

        base = base_model
        model = get_full_classification_model(base)

        Train(model, size, name)     
        save_model_to_hf(model, name)
        
print("\nAll runs completed.")

### Classification head Fine-Tuning
Run execution on small, medium and full dataset. Save results on HF platform

In [ ]:
for size in ['small', 'medium', 'full']:
        name = f"head_fine_tuning_set_{size}"

        base = base_model
        model = get_classification_head_model(base)

        Train(model, size, name)     
        save_model_to_hf(mode, name)
print("\nAll runs completed.")

### Adapters  Fine-Tuning
Run execution on small, medium and full dataset for different inner dimension adapters size. Save results on HF platform

In [ ]:
ADAPTERS_SIZE = [
    32, 64, 128
]

In [ ]:
for adapter_size in ADAPTERS_SIZE:
    for size in ['small', 'medium', 'full']:
        name = f"adapters_a_size_{adapter_size}_set_{size}"

        base = base_model
        model = get_adapters_model(base, adapter_size=adapter_size)

        Train(model, size, name)     

print("\nAll runs completed.")

### LoRA  Fine-Tuning
Run execution on small, medium and full dataset for different hyperparameters. Save results on HF platform

In [ ]:
LORA_CONFIGS = [
    {"r": 4, "alpha": 2},
    {"r": 8, "alpha": 4},
    {"r": 32, "alpha": 16},
    {"r": 4, "alpha": 8},
    {"r": 8, "alpha": 16},
    {"r": 32, "alpha": 64},
]

In [ ]:
for cfg in LORA_CONFIGS:
    for size in ['small', 'medium', 'full']:
        r = cfg['r']
        alpha = cfg['alpha']
        name = f"lora_r_{r}_alpha_{alpha}_set_{size}"

        base = base_model
        model = get_lora_model(base, rank=r, alpha=alpha)

        Train(model, size, name)     

print("\nAll runs completed.")